In [1]:
import matplotlib.pyplot as plt
import numpy as np
from os import mkdir
from os.path import join

In [2]:
bov_counter = 0

def writeBOV(g):
    """g is presumed to be a numpy 2D array of doubles"""
    global bov_counter
    bovNm = 'file_%03d.bov' % bov_counter
    dataNm = 'file_%03d.doubles' % bov_counter
    bov_counter += 1
    try:
        mkdir('frames')
    except FileExistsError:
        pass
    with open(join('frames', bovNm), 'w') as f:
        f.write('TIME: %g\n' % float(bov_counter))
        f.write('DATA_FILE: %s\n' % dataNm)
        f.write('DATA_SIZE: %d %d 1\n' % g.shape)
        f.write('DATA_FORMAT: DOUBLE\n')
        f.write('VARIABLE: U\n')
        f.write('DATA_ENDIAN: LITTLE\n')
        f.write('CENTERING: ZONAL\n')
        f.write('BRICK_ORIGIN: 0. 0. 0.\n')
        f.write('BRICK_SIZE: 1.0 1.0 1.0\n')
    with open(join('frames', dataNm), 'w') as f:
        g.T.tofile(f)  # BOV format expects Fortran order


In [3]:
#
# Scaling constants
#
# You'll have to pick a value for dt which produces stable evolution
# for your stencil!

XDIM = 101
YDIM = 101
tMax = 5.0
dx = 0.1
dy = 0.1
dt = 0.025 # FIX ME! 
vel = 1.0
xMin = -(XDIM//2)*dx
yMin = -(YDIM//2)*dy

In [4]:
def initialize():
    """Create the grid and apply the initial condition"""
    U = np.zeros([YDIM, XDIM])  # We just use this for shape

    ctrX= 0.0
    ctrY= 0.0
    sigma= 0.25
    maxU= 5.0

    grid = np.indices(U.shape)
    x = (grid[1] * dx) + xMin  # a full grid of X coordinates
    y = (grid[0] * dy) + yMin  # a full grid of Y coordinates
    distSqr = np.square(x - ctrX) + np.square(y - ctrY)
    U = maxU * np.exp(-distSqr/(sigma*sigma))
    
    return U

In [5]:
# test writeBOV
bov_counter = 0
writeBOV(initialize())

In [6]:
def doTimeStep(U, UOld):
    """
    Step your solution forward in time.  You need to calculate
    UNew in the grid area [1:-1, 1:-1].  The 'patch the boundaries'
    bit below will take care of the edges at i=0, i=XDIM-1, j=0,
    and j=YDIM-1.  Note that the array indices are ordered like U[j][i]!
    """

    xRatioSqr= (dt*dt*vel*vel)/(dx*dx)
    yRatioSqr= (dt*dt*vel*vel)/(dy*dy)

    UNew = np.empty_like(U)

    dxxterm = xRatioSqr * (U[1:-1, 2:] + U[1:-1, 0:-2] - 2*U[1:-1, 1:-1])
    dyyterm = yRatioSqr * (U[2:, 1:-1] + U[0:-2, 1:-1] - 2*U[1:-1, 1:-1])
    UNew[1:-1, 1:-1] = 2*U[1:-1,1:-1] + (dxxterm + dyyterm) - UOld[1:-1, 1:-1]

    # Patch the boundaries.  This mapping makes the surface into a torus.
    UNew[:, 0] = UNew[:, 1]
    UNew[:, -1] = UNew[:, -2]
    UNew[0, :] = UNew[1, :]
    UNew[-1, :] = UNew[-2, :]

    return UNew

In [7]:
def timeToOutput(t, count):
    """A little test to tell how often to dump output"""
    return (count % 4 == 0)

In [8]:

U = initialize()

UOld = np.copy(U)

t = 0.0
count = 0
while t < tMax:
    if timeToOutput(t, count):
        writeBOV(U)
        print ('Output at t = %s: min = %f, max = %f'
               % (t, np.amin(U), np.amax(U)))
    UNew = doTimeStep(U, UOld)
    UOld = U
    U = UNew
    t += dt
    count += 1

Output at t = 0.0: min = 0.000000, max = 5.000000
Output at t = 0.1: min = 0.000000, max = 3.333759
Output at t = 0.19999999999999998: min = 0.000000, max = 1.405772
Output at t = 0.3: min = -1.280158, max = 1.176781
Output at t = 0.4000000000000001: min = -1.505998, max = 1.075271
Output at t = 0.5000000000000001: min = -1.055168, max = 0.994922
Output at t = 0.6000000000000002: min = -0.836015, max = 0.918153
Output at t = 0.7000000000000003: min = -0.715695, max = 0.872852
Output at t = 0.8000000000000004: min = -0.641914, max = 0.809942
Output at t = 0.9000000000000005: min = -0.588482, max = 0.782391
Output at t = 1.0000000000000004: min = -0.546351, max = 0.741648
Output at t = 1.1: min = -0.515050, max = 0.702527
Output at t = 1.1999999999999997: min = -0.486513, max = 0.685882
Output at t = 1.2999999999999994: min = -0.467636, max = 0.648079
Output at t = 1.399999999999999: min = -0.451008, max = 0.633354
Output at t = 1.4999999999999987: min = -0.436726, max = 0.609990
Output 